In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [2]:
df = pd.read_csv("cleaned_data.csv")

In [3]:
df.head()

,Unnamed: 0,total_sqft,bath,balcony,price,site_location,bhk
0,0,1056.0,2.0,1.0,39.07,Alandi Road,2
1,1,2894.0,4.0,1.0,245.00,Alandi Road,4
2,2,1084.0,2.0,2.0,50.00,Alandi Road,2
3,3,1230.0,2.0,2.0,80.00,Alandi Road,2
4,4,1750.0,3.0,2.0,130.00,Alandi Road,3


In [4]:
df.drop(columns='Unnamed: 0', inplace=True)

In [5]:
x = df.drop(columns='price')
y = df['price']

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

In [7]:
x_train.shape

(5872, 5)

In [8]:
x_test.shape

(1469, 5)

In [9]:
x_train[:10]

,total_sqft,bath,balcony,site_location,bhk
268,1875.0,3.0,3.0,Aundh,3
7018,800.0,2.0,1.0,Wagholi,2
6552,1785.0,3.0,2.0,Tilak Road,3
5794,1075.0,2.0,1.0,Sasson Road,2
4350,1520.0,2.0,3.0,Model colony,3
126,1000.0,2.0,1.0,Ambegaon Budruk,2
6654,1191.0,2.0,1.0,Uruli Devachi,2
3818,5000.0,5.0,3.0,Law College Road,4
5071,1257.0,2.0,1.0,Pashan,2
3593,1105.0,2.0,1.0,Kondhwa Khurd,2


In [10]:
y_train[:10]

268     110.0
7018     25.0
6552     94.0
5794     52.0
4350     81.0
126      68.0
6654     90.0
3818    300.0
5071    105.0
3593     80.0
Name: price, dtype: float64

In [11]:
column_trans = make_column_transformer((OneHotEncoder(sparse_output=False), ['site_location']), remainder='passthrough')

In [12]:
scaler = StandardScaler()

In [13]:
lr = LinearRegression()

In [14]:
pipe = make_pipeline(column_trans,scaler,lr)

In [15]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['site_location'])])),
                ('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])

In [16]:
y_pred_lr = pipe.predict(x_test)

In [17]:
r2_score(y_test, y_pred_lr)

0.8520024671216193

In [19]:
data = [2345, 2, 2, 'Wagholi', 2]
data = pd.Series(data)

print(type(data))
data = data.values.reshape(1,5)

<class 'pandas.core.series.Series'>


In [18]:
import pickle

pickle.dump(pipe, open('LinearModel.pkl', 'wb'))

In [23]:
df = pd.DataFrame(data, columns=['total_sqft', 'bath', 'balcony', 'site_location', 'bhk'])
df

,total_sqft,bath,balcony,site_location,bhk
0,2345,2,2,Wagholi,2


In [32]:
prediction = pipe.predict(df)
print("Price for your house should be: ", round(prediction[0]*100000),"")

Price for your house should be:  14269088 


In [33]:
import pandas as pd
import streamlit as st
import pickle
import numpy as np

data = pd.read_csv('cleaned_data.csv')
pipe = pickle.load(open('LinearModel.pkl', 'rb'))

def predict_price(sqft, bath, balcony, location, bhk):
    input_data = pd.DataFrame([[sqft, bath, balcony, location, bhk]], columns=['total_sqft', 'bath', 'balcony', 'site_location', 'bhk'])
    prediction = pipe.predict(input_data)[0] * 1e5
    return np.round(prediction, 2)


st.title("Pune House Price Predictor")

# Get unique locations from the data
locations = sorted(data['site_location'].unique())

# Input components
location = st.selectbox("Select Location", locations)
bhk = st.text_input("Enter BHK", "")
sqft = st.text_input("Enter total house area in sqft", "")
bath = st.text_input("Enter number of bathroom(s)", "")
balcony = st.text_input("Enter number of balcony(ies)", "")

# Predict button
if st.button("Predict Price"):
    if location and bhk and sqft and bath and balcony:
        prediction = predict_price(float(sqft), int(bath), int(balcony), location, int(bhk))
        st.write(f"Prediction: ₹{prediction}")
    else:
        st.warning("Please fill in all the input fields.")

2024-02-09 13:01:56.875 
  command:

    streamlit run C:\Users\Shardul\AppData\Local\Programs\Python\Python312\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


SyntaxError: invalid syntax (2305678394.py, line 1)